<a href="https://colab.research.google.com/github/abbassix/life-simulation/blob/main/LifeSim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
number_of_rows = 10
number_of_columns = 10
number_of_entities = 10
fact_book = {}

In [3]:
def update_fact_book(entity, location, creation_time=None):
  global fact_book
  if entity not in fact_book.keys():
    fact_book[entity] = {}
  if location:
    fact_book[entity]['location'] = location
  if creation_time != None:
    fact_book[entity]['creation_time'] = creation_time

In [4]:
def genesis(number_of_rows, number_of_columns, number_of_entities):
  time_sapce = np.zeros((1, number_of_rows, number_of_columns))
  entities = [n+1 for n in range(number_of_entities)]
  while len(entities):
    fiat_row = np.random.randint(number_of_rows)
    fiat_column = np.random.randint(number_of_columns)
    if time_sapce[0, fiat_row, fiat_column] == 0:
      update_fact_book(entity=entities[0], location=[fiat_row, fiat_column], creation_time=0)
      time_sapce[0, fiat_row, fiat_column] = entities[0]
      del entities[0]
  return(time_sapce)

In [5]:
time_sapce = genesis(number_of_rows, number_of_columns, number_of_entities)
time_sapce

array([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  9.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,  0.],
        [ 0.,  0.,  0.,  4.,  0.,  8.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  5.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  2.,  0.],
        [ 0.,  0.,  0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  7.,  0.,  0.,  0.,  6.,  0.,  0.,  0.,  0.]]])

In [6]:
fact_book

{1: {'creation_time': 0, 'location': [7, 4]},
 2: {'creation_time': 0, 'location': [7, 8]},
 3: {'creation_time': 0, 'location': [8, 3]},
 4: {'creation_time': 0, 'location': [2, 3]},
 5: {'creation_time': 0, 'location': [4, 9]},
 6: {'creation_time': 0, 'location': [9, 5]},
 7: {'creation_time': 0, 'location': [9, 1]},
 8: {'creation_time': 0, 'location': [2, 5]},
 9: {'creation_time': 0, 'location': [0, 8]},
 10: {'creation_time': 0, 'location': [1, 7]}}

In [7]:
def next_(time_sapce, time):
  # create new time-layer IF there isn't a.r.
  time_sapce = np.append(time_sapce, time_sapce[time:,:,:], axis=0)  # create new time-layer
  return time_sapce

In [8]:
time_sapce = next_(time_sapce, time=0)
time_sapce

array([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  9.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,  0.],
        [ 0.,  0.,  0.,  4.,  0.,  8.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  5.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  2.,  0.],
        [ 0.,  0.,  0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  7.,  0.,  0.,  0.,  6.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  9.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,  0.],
        [ 0.,  0.,  0.,  4.,  0.,  8.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  5.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,

In [9]:
def movement(time_sapce, entity, time, origin, destination):
  if abs(origin[0]-destination[0]) + abs(origin[1]-destination[1]) > 2:
    raise Exception('teleportation not possible')
  elif time_sapce[time, origin[0], origin[1]] != entity:
    raise Exception('nothing ab initio')
  elif time_sapce[time+1, destination[0], destination[1]] == 0:
    update_fact_book(entity=entity, location=[destination[0], destination[1]])
    time_sapce[time+1, origin[0], origin[1]] = 0
    time_sapce[time+1, destination[0], destination[1]] = entity
    return time_sapce
  else:
    raise Exception('conflict')

In [10]:
time_sapce = movement(time_sapce, entity=1, time=0, origin=[7, 4], destination=[6, 4])
time_sapce

array([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  9.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,  0.],
        [ 0.,  0.,  0.,  4.,  0.,  8.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  5.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  2.,  0.],
        [ 0.,  0.,  0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  7.,  0.,  0.,  0.,  6.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  9.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,  0.],
        [ 0.,  0.,  0.,  4.,  0.,  8.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  5.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,

In [11]:
fact_book

{1: {'creation_time': 0, 'location': [6, 4]},
 2: {'creation_time': 0, 'location': [7, 8]},
 3: {'creation_time': 0, 'location': [8, 3]},
 4: {'creation_time': 0, 'location': [2, 3]},
 5: {'creation_time': 0, 'location': [4, 9]},
 6: {'creation_time': 0, 'location': [9, 5]},
 7: {'creation_time': 0, 'location': [9, 1]},
 8: {'creation_time': 0, 'location': [2, 5]},
 9: {'creation_time': 0, 'location': [0, 8]},
 10: {'creation_time': 0, 'location': [1, 7]}}